# Fantasy football, drafting based on relative opportunities

- toc: true
- branch: master
- badges: true
- comments: false
- categories: [personal, data science]

## Overview

In a fantasy sports league, the draft is about adding players to fill out your roster.
The complexity greatly increases when the positions are introduced. 
This post will try to cover relevant considerations for fantasy football, assuming some correctness in quantitative forecasts of player performance.

## Fantasy Football

In fantasy football, player performance is translated into points.
Despite scoring settings that can serve to balance certain positions, ultimately certain positions (like QB) tend to score a lot. 
To balance this and ensure teams don't solely draft overpowered positions like QB, there are constraints on your roster: you have limits on how many players of each position you can have (QB, RB, WR, TE, K, D/ST).

When drafting, one must ask "should I use my draft pick on a high-valued QB or a high-valued RB"? 

Ultimately, it's not as simple as picking the player with higher projected points.
A more appropriate question is, "what am I giving up if I spend a draft pick on a high-valued RB instead of a high-valued QB"?

Let's explore a hypothetical situation given the following players' points projections:

- QBs: [20, 19, 18, 17, 16, 15]
- RBs: [15, 12, 10, 9, 8, 7]

In these projections, observe how there's a steady decline in QB scores while there is a steep drop in RBs after the first 2 RBs.

In this situation, we have 3 league managers, we must (snake) draft 1 QB and 1 RB, and we have the first round first pick.
For simplicity, let's neglect injury considerations

## Naive approach
In the naive approach, all players may select based on highest-points available:

- We pick the 20pt-QB, manager 2 gets the 19pt-QB, manager 3 gets the 18pt-QB
- Next, according to roster constraints, we *must* draft RBs
- Manager 3 picks the 15pt-RB, manager 2 picks the 12pt-RB, and we pick the 10pt-RB
- Based off projections, we expect 30 points while managers 2 and 3 expect 31 and 33 points, respectively.

For us (first round first pick), we drafted ourselves into last place.
When we picked the 20pt-QB (who was only slightly better than the other QBs), we gave up the opportunity to pick the 15pt-RB (who was considerably better than the other RBs).
When manager 3's turn came, he/she had the opportunity to pick a just-as-good-as-our QB or a much-better-than-the-rest RB.
Picking a slightly-worse QB was only a small setback, while picking the top-RB was a huge gain.

When drafting, again, we have to consider the other positions and their relative opportunity.

## Drafting on opportunity

Let's try drafting based on relative opportunity, while the other managers still pursue the naive approach

- We pick the 15pt-RB. Relatively speaking, he is a whopping 3 points better than the next RB. Further, if we imagine the "worst case" where managers 2 and 3 draft RBs while we drafted a QB, we would be left with the 10pt RB: the 15pt-RB is 5 points better than our worst case scenario RB. Comparing against QBs, the best QB is only 1 point better than the next QB and 2 points better tahn the worst case QB.
- Following the naive approach, manager 2 picks the 20pt-QB, manager 3 picks the 19pt-QB.
- For roster constraints, manager 3 picks the 12pt-RB and manager 2 picks the 10pt-RB.
- We pick the 18pt-QB (the best available QB)
- Our expected points come out to 33 points, while managers 2 and 3 project to 30 and 31 points, respectively.

Because we went with the RB who was considerably above his peers, we found ourselves with the best team.

## Two teams drafting on opportunity

Let's say we (manager 1) and manager 2 decide to draft based on relative opporunity while manager 3 still takes the greedy approach

- We pick the 15pt-RB, he is 3 points better than the next RB and 5points better than the worst case RB
- Manager 2 picks the 12pt-RB, he is 2 points better than the next RB (and also 2 points better than the worst-case RB)
- Manager 3 picks the 20pt QB then the 10pt RB
- Manager 2 picks the 19 pt QB. We pick the 18pt QB.
- Our total is 33 points, manager 2 has 31 points, and manager 3 has 30 points.
- In this outcome, we still have the best team, but now manager 2 projects higher than manager 3.

## Exercise summary

We can see that drafting on relative opportunity (or positional advantage if you want to call it), can optimize our draft outcomes.
In absolute numbers, a QB might be better than a RB.
In relative numbers (against their positional peers), that same QB might only be mildly better than the competition while the same RB is significantly ahead of his peers

## Real data discussion

We will be taking some data from [football absurdity](https://footballabsurdity.com/draft-sheet-form/?teams=10&bn=8&qb=1&rb=2&wr=2&te=1&qrwt=1&rwt=2&patd=4&rutd=6&retd=6&payd=0.04&ruyd=0.1&reyd=0.1&int=-1.0&rec=0.5&fum=-2.0).
This data assigns a "value" to each player (useful for auction drafts) based on league settings.
Regardless, the name of the game is still to maximize our team's overall value.
You could imagine substituting this data with some other data that assigns a fantasy estimate to each player.

Unlike our toy example, we cannot simply enumerate the extensive draft strategy variance in a 12-team league. 
In these situations, people like to think in terms of "tiers of players" -- if I miss out on drafting a tier 1 RB in favor of a tier 1 WR, what could the next RB look like (tier 2 or 3?).
In a rough estimation of a "draft simulation", you could compute relative opportunity based on what the next-best-player might look like (RB1 versus RB2), or the worst case scenario (if you draft first pick with RB1 and everyone else in the snake draft picks RBs, you might be comparing RB1 versus RB23), or some combination of the two ideas.
Then, compare that RB relative opportunity against the respective WR relative opportunity.
The larger relative opportunity should be used to draft the optimal player.

In the plot below, we have plotted the players and their values, sorted by rank and colored by position.
This should help to visualize the positional landscape.

In [3]:
import pandas as pd
import altair as alt

# Load some data
df = pd.read_csv("files/2023footballsheet.csv")

# Construct (interactive) altair chart
selector_legend = alt.selection_point(fields=["Position"], bind="legend")
(
    alt.Chart(
        df, 
        height=400, width=400
    )
    .mark_line(point=True)
    .encode(
        x=alt.X("Rank:Q").scale(
            domain=(
                df["Rank"].min() - 1, 
                df["Rank"].max() + 1
            )
        ),
        y=alt.Y("Pts/week:Q").scale(
            domain=(
                df["Pts/week"].min() - 1, 
                df["Pts/week"].max() + 1
            )
        ),
        color=alt.Color("Position:N"),
        tooltip=["Name", "Rank", "Pts/week"],
        opacity=alt.condition(selector_legend, alt.value(1), alt.value(0.2))
    )
    .add_params(selector_legend)
    .configure_point(size=50)
)

alt.Chart(...)

### Positional observations

- QBs: While the value of the first couple QBs is high, there's a steep dropoff every 2-ish QBs
- RBs: RB1&2 are significantly higher than the others, while RB3-8 are all similar, then a strong dropoff to RBs3-11, then a steady dropoff from there.
- WRs: WR1-4 are similar, strong dropoffs from WR5-13. WR13-20 are prety similar, and so are WR20-30 or so.
- TEs: TE1 is miles ahead of all the others. Just looking at the slope, there is a huge relative opportunity with Travis Kelce as your TE.

### Single QB vs 2QB (superflex) league

In single QB leagues, you are only allowed to play 1 QB per week.
In this situation, we can (generally) assume people will only draft 1 QB, so even though the NFL has so many teams and quarterbacks, our search space is confined to just QB1-12. 
If you have the opportunity to draft QB1 (Mahomes at 7.7 value), your worst case would end up QB12 (Jones at 2.6 value).
This is a relative opportunity of 5.1

However, if you are in a 2QB (superflex) league, your worst case actually becomes QB23 (Purdy at -0.6 value).
In this situation, the relative opportunity has inflated to 8.3.
If you miss on drafting a top QB, there will be stronger consequences compared to the single QB league.

### WR
From WR1-11, there are steep dropoffs to be found.
On the other hand, WR12-onwards is a fairly steady decline.

If you're deciding between a WR12+ and another position, you likely don't stand to miss much if you end up with WR24 versus WR18 (or something similar). In this situation, you can direct your focus to other positions

For WR1-11, because there are steep dropoffs, it helps to draft a top11 WR.

### RB
RB1-2 are their own tier, then RB3-8 their own tier, steepness RB9-11, and a steady decline RB12+ (but steeper than the WR12+ decline) onwards.

This means there should be a prioritization of RB1-2.
RB3-8 are comparable so it's not completely necessary to draft Henry over Chubb, perhaps it's more useful to dedicate your attention elsewhere if you know you will end up with anything RB3-8.

### WR vs RB

But there's steepness all over the place for WR and RB! I can't just simultaneously prioritze WRs and RBs!

NFL trends aside, it's important to consider the available player pool when picking between drafting WR vs RB.

To choose when to pick a WR or RB, again revisit the relative opportunity of:
- The better WR versus worst case WR and the better RB versus worst case RB. 
    - This approach makes some strong assumptions about how the rest of the draft will play out, so you may be factoring in too many cliffs when looking at the relative opportunity
- The better WR versus next-bset WR and the better RB versus next-best RB.
    - This approach is rather near-sighted because it assumes you'd have the next-best player available, which is probably not the case in a fantasy league, so you may be factoring in too few cliffs.
- The choice of approach initially seems up to personal preference (near-sighted versus far-sighted but lots of assumptions). Perhaps a simple average of the relative opportunities of the two approaches yields a decent estimate for the "true" relative opportunity

### TE

Considering most leagues are single TE, we consider the range of TE1-12.
Between TE2-12, there's a steady decline in quality.
With no steep dropoffs to be found, there is no "prime opportunity" to pick a certain TE.
TE1 (Kelce at 7.4) is projected to be significantly better than all other TEs. 
This TE1 versus TE2-12 steepness is a great example of relative opportunity.
Even if TE1 won't score as many points as some QBs, TE1 will score way more than other TEs.


## When others zig, you zag

In terms of "draft principles", here's another one.
From the plots, we can see that the top X players per position show strong steepness (relatively much better than their peers) followed by some flatter decay (no significant difference among players).

If others in your draft appear to be prioritizing WRs, it means your league is quickly approaching the WR flatline and WRs are effectively indistinguishable from one another.
This, however, exposes the top-end RB steepeness that no one has touched.
Calculations aside, WRs are hitting flatland while RBs are still on steep decline, so it could be a good idea to draft RBs when others are drafting WRs.